In [511]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/kalshi_trade_data_2025.csv')

In [3]:
contract_wise = df.groupby('ticker_name').agg({'contracts_traded': 'sum', 'price': 'mean'})

In [4]:
contract_wise = contract_wise.sort_values('contracts_traded', ascending=False)

In [8]:
contract_wise = contract_wise.reset_index()

In [184]:
contract_wise['ticker_only'] = contract_wise['ticker_name'].apply(lambda x: x.split('-')[0])

In [ ]:
category = json.loads(open('categories.json').read())

# Method 1: Using apply with a function
def categorize_ticker(ticker_name):
    for key in category.keys():
        if key in str(ticker_name):
            return category[key]
    return 'Uncategorized'

contract_wise['category'] = contract_wise['ticker_name'].apply(categorize_ticker)

In [508]:
contract_wise.to_csv('data/category_wise.csv', index=None)

In [509]:
null = contract_wise[contract_wise['category'] == 'Uncategorized']

In [510]:
null['contracts_traded'].sum() / contract_wise['contracts_traded'].sum()

0.010726081771927699

In [500]:
len(null)/len(contract_wise)

0.008070637349083233

In [501]:
null[null['ticker_name'].str.contains('KXVMAKPOP')]

,ticker_name,contracts_traded,price,category,ticker_only


In [493]:
null['ticker_only'].value_counts()

ticker_only
KXENDORSECUOMO          5
KXAMAFCS                5
KXNEWPOPEDATE           5
KXBCHMINY               5
KXEMMYRCPROGRAM         5
                       ..
POPVOTEMOVNV            1
KX1SONGOLIVIARODRIGO    1
KXOAIMUSKSETTLE         1
KXMUSKINJ               1
KXPARLITURKEY           1
Name: count, Length: 1718, dtype: int64

In [ ]:
df = pd.read_csv('data/kalshi_trade_data_2025.csv')
category = json.loads(open('categories.json').read())
pattern = '|'.join(category.keys())


def categorize_chunk(chunk):
    global pattern
    return chunk['ticker_name'].str.findall(pattern).str[0].map(category)

# Process in chunks
chunk_size = 100000
chunks = []
for i in range(0, len(df), chunk_size):
    print(f"{round(i/len(df) * 100, 2)}% done")
    chunk = df.iloc[i:i+chunk_size]
    chunk['category'] = categorize_chunk(chunk)
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
df.to_csv('data/df_with_category.csv', index=None)

In [515]:
df.columns

Index(['create_ts', 'ticker_name', 'contracts_traded', 'price'], dtype='object')